<a href="https://colab.research.google.com/github/mangalm96/explainablenlp-info259/blob/main/process_tWTWT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ekphrasis

     |████████████████████████████████| 80 kB 4.1 MB/s 
     |████████████████████████████████| 45 kB 2.8 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-py3-none-any.whl size=82842 sha256=b28d369cf5b43fef67881619a459ab92947c17415a1f6345d9f3b275e207f039
  Stored in directory: /root/.cache/pip/wheels/f7/ec/0d/12659e32faf780546945d0120f2c8410eb3efb7426731da88f
Successfully built ekphrasis


In [2]:
import json
import os
import sys
import glob

import re
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [3]:
EMOJI_PATTERN = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [4]:
text_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone',
        'time', 'url', 'date', 'number'],
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},

    fix_html=True,
    segmenter="twitter",
    corrector="twitter",

    unpack_hashtags=True,
    unpack_contractions=True,
    spell_correct_elong=True,

    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    dicts=[emoticons]
)


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [5]:
REMOVE_TAGS = [
    "<emphasis>", "<kiss>", "<repeated>", "<laugh>", "<allcaps>",
    "</allcaps>", "<angel>", "<elongated>", "<tong>", "<annoyed>",
    "<censored>", "<happy>", "<percent>", "<wink>",
    "<headdesk>", "<surprise>", "<date>", "<time>", "<url>",
    "<sad>", "<email>", "<phone>", "<hashtag>", "</hashtag>"
    ]

ADD_TO_GLOVE = ["<number>", "<money>"]

In [6]:
REMOVE_PUNCTUATIONS = ["!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "-", "+", "_", "=", "{", "}", "[", "]", ":", ";", "'", '"', ",", ".", "?", "/"]

In [7]:
# Try removing punctuations as well

def pre_process_single(tweet, t_id):
    tweet_toked_text = []
    de_emojified_text = tweet.encode('ascii', 'ignore').decode('ascii')
    de_emojified_text = re.sub(r"\$", " $ ", de_emojified_text)
    company_normalize_text = EMOJI_PATTERN.sub(r' ', de_emojified_text)

    tokens = text_processor.pre_process_doc(company_normalize_text)
    for token in tokens:
        if token in REMOVE_TAGS:
            continue
        if token in REMOVE_PUNCTUATIONS: #remove punctuations
            continue
        tweet_toked_text.append(str(token))

    return tweet_toked_text

In [8]:
id2text = {}
tweet_acquired = 0
tweet_valid = 0
tweet_invalid = 0

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
from zipfile import ZipFile
with ZipFile("/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/data/wtwt/scrapped_full.zip", 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

Streaming output truncated to the last 5000 lines.
scrapped_full/949435169630547968.json          2022-04-01 05:35:10          337
scrapped_full/956858286359408641.json          2022-04-01 18:35:50          287
scrapped_full/590593286835982338.json          2022-03-31 09:27:52          229
scrapped_full/911097017061056517.json          2022-04-01 02:47:18          236
scrapped_full/1069992225113747456.json         2022-03-31 08:19:28          197
scrapped_full/940219947778748417.json          2022-03-30 23:06:44          149
scrapped_full/939160814547755011.json          2022-03-30 21:38:32          191
scrapped_full/831684766021844992.json          2022-03-31 22:14:24          198
scrapped_full/1058794081835511809.json         2022-03-31 08:01:40          226
scrapped_full/857989117816299520.json          2022-04-01 01:06:04          228
scrapped_full/929210863273283584.json          2022-04-01 18:52:32          189
scrapped_full/956316958751903744.json          2022-04-01 04:11:12   

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Done!


In [11]:
path = 'scrapped_full/'
files = os.listdir(path)
files # all files are here

['912255772947550209.json',
 '931422083506229248.json',
 '931330653890625537.json',
 '762755281423994880.json',
 '1037844398413938689.json',
 '1059807538772873216.json',
 '1027936125900476416.json',
 '627849121609596928.json',
 '927295003532992514.json',
 '537282405099667456.json',
 '928141225919524865.json',
 '955700478465249280.json',
 '983473494586376192.json',
 '471378255300558848.json',
 '804173177903521792.json',
 '971771856989708288.json',
 '979510479860019200.json',
 '912773060822028294.json',
 '1064987510747611136.json',
 '985057083950116864.json',
 '827926014554673157.json',
 '923958803896389634.json',
 '618409495027281921.json',
 '941743896534556672.json',
 '970755681048506369.json',
 '912818890903826432.json',
 '1053298332771254273.json',
 '612997700074606592.json',
 '944151049845268480.json',
 '925125148918808576.json',
 '1017604666249621505.json',
 '937396896733253632.json',
 '945753593587658758.json',
 '542924294628835328.json',
 '974670515540545539.json',
 '976531436466

In [13]:
files[0]

'912255772947550209.json'

In [12]:
for fil in files:
    file_name = path + fil
    # print(file_name)
    if fil!=".DS_Store":
      fo = open(file_name, "r")
      # print(fo)
      full_tweet = json.load(fo)
      fo.close()
      tweet_acquired = tweet_acquired + 1
      if "data" not in full_tweet:
          tweet_invalid = tweet_invalid + 1
          continue
      tweet_valid = tweet_valid + 1
      tweet_id = full_tweet["data"][0]["id"]
      txt = pre_process_single(full_tweet["data"][0]["text"], tweet_id)
      if len(txt) > 0 and txt != ["<user>"]:
          id2text[tweet_id] = txt
      else: 
          pass

In [16]:
print(f'the number of tweet acquired is:',{tweet_acquired})
print(f'the number of invalid tweet is:',{tweet_invalid})
print(f'the number of valid tweet is:',{tweet_valid})

the number of tweet acquired is: {50208}
the number of invalid tweet is: {7190}
the number of valid tweet is: {43018}


In [17]:
all_keys = id2text.keys()
len(all_keys)

43010

In [18]:
fo = open("/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/data/twt_wt/wtwt_new.json", "r")
wtwt = json.load(fo)
fo.close()

In [19]:
wtwt_obtained = []

In [20]:
merger2target = json.load(open('/content/gdrive/MyDrive/NLP Stance Detection/tWT_WT/dataset/merger2target.json'))

In [21]:
for data in wtwt:
    if data["tweet_id"] in all_keys:
        d = data.copy()
        d["target"] = merger2target[d["merger"]].split()
        d["text"] = id2text[data["tweet_id"]]
        d["merger"] = d["merger"][4:] if "NEG_" in d["merger"] else d["merger"]
        assert d["merger"] in ['CVS_AET', 'CI_ESRX', 'AET_HUM', 'ANTM_CI', 'DIS_FOX']
        wtwt_obtained.append(d)

In [22]:
fo = open("/content/gdrive/MyDrive/NLP Stance Detection/tWT_WT/dataset/data_new2.json", "w+")
json.dump(wtwt_obtained, fo, indent=2)
fo.close()

In [ ]:
## checking

import json
data = open("/content/gdrive/MyDrive/NLP Stance Detection/tWT_WT/dataset/data_new.json")
data_list = json.load(data)

In [ ]:
for k in data_list:
  if k['tweet_id'] == '971761970117357568' :
    print(k)
    print(k['stance'])

{'tweet_id': '971761970117357568', 'merger': 'CI_ESRX', 'stance': 'support', 'target': ['Cigna', 'buys', 'Express_Scripts'], 'text': ['Cigna', 'and', 'ESI', 'set', 'to', 'merge', 'Here', 'we', 'go']}
support
{'tweet_id': '971761970117357568', 'merger': 'CI_ESRX', 'stance': 'refute', 'target': ['Cigna', "won't", 'buy', 'Express_Scripts'], 'text': ['Cigna', 'and', 'ESI', 'set', 'to', 'merge', 'Here', 'we', 'go']}
refute


In [23]:
## checking

import json
data = open("/content/gdrive/MyDrive/NLP Stance Detection/tWT_WT/dataset/data_new2.json")
data_list = json.load(data)

In [26]:
i=1
for k in data_list:

  if k['tweet_id'] == '971761970117357568' :
    print(k)
    print("Number:",i)
    print(k['stance'])
  i+=1

{'tweet_id': '971761970117357568', 'merger': 'CI_ESRX', 'stance': 'support', 'target': ['Cigna', 'buys', 'Express_Scripts'], 'text': ['Cigna', 'and', 'ESI', 'set', 'to', 'merge', 'Here', 'we', 'go']}
Number: 19587
support
{'tweet_id': '971761970117357568', 'merger': 'CI_ESRX', 'stance': 'refute', 'target': ['Cigna', "won't", 'buy', 'Express_Scripts'], 'text': ['Cigna', 'and', 'ESI', 'set', 'to', 'merge', 'Here', 'we', 'go']}
Number: 85968
refute


In [ ]:
1031916968025251840

In [25]:
for k in data_list:
  if k['tweet_id'] == '1031916968025251840' :
    print(k)
    print(k['stance'])

{'tweet_id': '1031916968025251840', 'merger': 'CI_ESRX', 'stance': 'refute', 'target': ['Cigna', "won't", 'buy', 'Express_Scripts'], 'text': ['Icahn', 'Drops', 'Fight', 'to', 'Block', 'Cigna', 's', 'Express', 'Scripts', 'Takeover']}
refute
{'tweet_id': '1031916968025251840', 'merger': 'CI_ESRX', 'stance': 'support', 'target': ['Cigna', 'buys', 'Express_Scripts'], 'text': ['Icahn', 'Drops', 'Fight', 'to', 'Block', 'Cigna', 's', 'Express', 'Scripts', 'Takeover']}
support
